In [1]:
import pandas as pd
import numpy as np
import pickle
import inspect
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

# importing project modules
import helper_functions
import config
import model_trainer
import all_models

pd.options.display.max_columns = None
pd.options.display.max_rows = None

# functions_list = [func for func in dir(helper_functions) if inspect.isfunction(getattr(helper_functions, func))]
# functions_list

Required Input Features

In [3]:
pd_features = [ 'grade',
                'home_ownership',
                'addr_state',
                'verification_status',
                'purpose',
                'initial_list_status',
                'term',
                'emp_length',
                'issue_d',
                'int_rate',
                'earliest_cr_line',
                'inq_last_6mths',
                'acc_now_delinq',
                'annual_inc',
                'dti',
                'mths_since_last_delinq',
                'mths_since_last_record']

lgd_features =         [    'grade',
                            'home_ownership',
                            'verification_status',
                            'purpose',     
                            'initial_list_status',
                            'term',
                            'emp_length',
                            'issue_d',
                            'earliest_cr_line',
                            'funded_amnt',
                            'int_rate',
                            'installment',
                            'annual_inc',
                            'dti',
                            'delinq_2yrs',
                            'inq_last_6mths',
                            'mths_since_last_delinq',
                            'mths_since_last_record',
                            'open_acc',
                            'pub_rec',
                            'total_acc',
                            'acc_now_delinq',
                            'total_rev_hi_lim']

# Total no. of features that need to be entered.
total_features = list(set(pd_features).union(lgd_features))
print(len(total_features))
# total_features

24


PD Model Prediction

In [3]:
# this is for web based development
input_df =  helper_functions.customdata(
    grade = "A",
    issue_d = '2014-01-12',
    int_rate = 9,
    annual_inc = 768900
)
pred_dict = helper_functions.default_prediction(input_df)
pred_dict

{'default_prediction': array([1]),
 'default_prob': array([[0.02300605, 0.97699395]])}

LGD, EAD and EL

In [4]:
lgd_model_stage_1 = pickle.load(open(r'C:\Shubham\credit_scorecard\src\lgd_model_stage_1.sav',"rb"))
lgd_model_stage_2 = pickle.load(open(r'C:\Shubham\credit_scorecard\src\lgd_model_stage_2.sav',"rb"))
reg_ead = pickle.load(open(r'C:\Shubham\credit_scorecard\src\reg_ead.sav',"rb"))
reg_pd = pickle.load(open(r'C:\Shubham\credit_scorecard\src\lr_pd_model_with_pvalue_filtered_features.sav', 'rb'))

In [5]:
# storing input values of variables in a dictionary
var_data_dict = {
    'grade' : 'A',
    'home_ownership' : 'OWN',
    'addr_state' : 'NY',
    'verification_status' : 'Verifeid',
    'purpose' : 'education',
    'initial_list_status' : 'f',
    'term' : 36,
    'emp_length' : 2,
    'issue_d' : '2013-09-01',
    'int_rate' : 8,
    'earliest_cr_line' : '2005-09-01',
    'delinq_2yrs' : 2,
    'inq_last_6mths' : 3,
    'open_acc' : 18,
    'pub_rec' : 4,
    'total_acc' : 27,
    'acc_now_delinq' : 0,
    'total_rev_hi_lim' : 30000,
    'annual_inc' : 1000000,
    'dti' : 4,
    'mths_since_last_delinq' : 20,
    'mths_since_last_record' : 10,
    'funded_amnt' : 100000,
    'installment' : 6
}

In [10]:
from datetime import datetime

In [17]:
# predictions for a customer inputs
def final_predictions(var_data_dict):

    pd_features = [ 'grade',
                    'home_ownership',
                    'addr_state',
                    'verification_status',
                    'purpose',
                    'initial_list_status',
                    'term',
                    'emp_length',
                    'issue_d',
                    'int_rate',
                    'earliest_cr_line',
                    'inq_last_6mths',
                    'acc_now_delinq',
                    'annual_inc',
                    'dti',
                    'mths_since_last_delinq',
                    'mths_since_last_record']

    lgd_features =         [    'grade',
                                'home_ownership',
                                'verification_status',
                                'purpose',     
                                'initial_list_status',
                                'term',
                                'emp_length',
                                'issue_d',
                                'earliest_cr_line',
                                'funded_amnt',
                                'int_rate',
                                'installment',
                                'annual_inc',
                                'dti',
                                'delinq_2yrs',
                                'inq_last_6mths',
                                'mths_since_last_delinq',
                                'mths_since_last_record',
                                'open_acc',
                                'pub_rec',
                                'total_acc',
                                'acc_now_delinq',
                                'total_rev_hi_lim']

    pd_input_var_dict = {key: var_data_dict[key] for key in pd_features if key in var_data_dict}
    lgd_input_var_dict = {key: var_data_dict[key] for key in lgd_features if key in var_data_dict}

    # customizing the input var values
    ## fro PD
    pd_input_df = helper_functions.customize_data_pd(pd_input_var_dict)
    ## for LGD, EAD and EL
    lgd_input_var_dict['term_int'] = lgd_input_var_dict.pop('term')
    lgd_input_var_dict['emp_length_int'] = lgd_input_var_dict.pop('emp_length')
    lgd_input_var_dict['mths_since_issue_d'] = lgd_input_var_dict.pop('issue_d')
    issue_date = datetime.strptime(lgd_input_var_dict['mths_since_issue_d'], "%Y-%m-%d")
    end_date = datetime.strptime('2017-12-01', "%Y-%m-%d")
    lgd_input_var_dict['mths_since_issue_d'] = end_date.year * 12 + end_date.month - (issue_date.year * 12 + issue_date.month)
    lgd_input_var_dict['mths_since_earliest_cr_line'] = lgd_input_var_dict.pop('earliest_cr_line')
    issue_date = datetime.strptime(lgd_input_var_dict['mths_since_earliest_cr_line'], "%Y-%m-%d")
    end_date = datetime.strptime('2017-12-01', "%Y-%m-%d")
    lgd_input_var_dict['mths_since_earliest_cr_line'] = end_date.year * 12 + end_date.month - (issue_date.year * 12 + issue_date.month)
    lgd_input_df = helper_functions.customize_data_lgd(lgd_input_var_dict)

    # loading models
    lgd_model_stage_1 = pickle.load(open(r'C:\Shubham\credit_scorecard\src\lgd_model_stage_1.sav',"rb"))
    lgd_model_stage_2 = pickle.load(open(r'C:\Shubham\credit_scorecard\src\lgd_model_stage_2.sav',"rb"))
    reg_ead = pickle.load(open(r'C:\Shubham\credit_scorecard\src\reg_ead.sav',"rb"))
    reg_pd = pickle.load(open(r'C:\Shubham\credit_scorecard\src\lr_pd_model_with_pvalue_filtered_features.sav', 'rb'))

    # make predictions
    loan_data_preprocessed_lgd_ead = lgd_input_df.copy()
    loan_data_preprocessed = loan_data_preprocessed_lgd_ead.copy()

    loan_data_preprocessed = helper_functions.expected_loss_estimation(lgd_model_stage_1,lgd_model_stage_2,reg_ead,loan_data_preprocessed,loan_data_preprocessed_lgd_ead)

    # predicting default for a customer
    def default_prediction(input_df,model):

        pred_dict = {
            "default_prediction" : model.predict(input_df),
            "default_prob" : model.predict_proba(input_df)
        }

        return pred_dict

    pred_pd = default_prediction(pd_input_df,reg_pd)
    print(pred_pd)
    print(pred_pd['default_prob'][0][1])
    print(loan_data_preprocessed.columns)
    loan_data_preprocessed['PD'] = pred_pd['default_prob'][0][1]
    loan_data_preprocessed['EL'] = loan_data_preprocessed['PD'] * loan_data_preprocessed['LGD'] * loan_data_preprocessed['EAD']
    loan_data_preprocessed['EL/funded_amnt'] = loan_data_preprocessed['EL']/loan_data_preprocessed['funded_amnt']
    
    # creating dictionary of predicted values
    pred_values = {
        "Prob. of Default" : loan_data_preprocessed['PD'][0],
        "Recovery Rate" : loan_data_preprocessed['recovery_rate'][0],
        "Loss Given Default" : loan_data_preprocessed['LGD'][0],
        "Credit Conversion Factor" : loan_data_preprocessed['CCF'][0],
        "Exposure at Default" : loan_data_preprocessed['EAD'][0],
        "Expected Loss" : loan_data_preprocessed['EL'][0],
        "EL/Funded Amount" : loan_data_preprocessed['EL/funded_amnt'][0],
    }

    return pred_values

In [18]:
pred_values = final_predictions(var_data_dict)

{'default_prediction': array([1]), 'default_prob': array([[0.03368801, 0.96631199]])}
0.9663119917365041


AttributeError: 'NoneType' object has no attribute 'columns'

In [9]:
# splitting data wrt models
## for PD Model:
pd_input_var_dict = {key: var_data_dict[key] for key in pd_features if key in var_data_dict}
print(len(pd_features))
print(len(pd_input_var_dict.keys()))
## for LGD, EAD and EL
lgd_input_var_dict = {key: var_data_dict[key] for key in lgd_features if key in var_data_dict}
print(len(lgd_features))
print(len(lgd_input_var_dict.keys()))

17
17
23
23


In [10]:
from datetime import datetime
# customizing the input var values
## fro PD
pd_input_df = helper_functions.customize_data_pd(pd_input_var_dict)
## for LGD, EAD and EL
lgd_input_var_dict['term_int'] = lgd_input_var_dict.pop('term')
lgd_input_var_dict['emp_length_int'] = lgd_input_var_dict.pop('emp_length')
lgd_input_var_dict['mths_since_issue_d'] = lgd_input_var_dict.pop('issue_d')
issue_date = datetime.strptime(lgd_input_var_dict['mths_since_issue_d'], "%Y-%m-%d")
end_date = datetime.strptime('2017-12-01', "%Y-%m-%d")
lgd_input_var_dict['mths_since_issue_d'] = end_date.year * 12 + end_date.month - (issue_date.year * 12 + issue_date.month)
lgd_input_var_dict['mths_since_earliest_cr_line'] = lgd_input_var_dict.pop('earliest_cr_line')
issue_date = datetime.strptime(lgd_input_var_dict['mths_since_earliest_cr_line'], "%Y-%m-%d")
end_date = datetime.strptime('2017-12-01', "%Y-%m-%d")
lgd_input_var_dict['mths_since_earliest_cr_line'] = end_date.year * 12 + end_date.month - (issue_date.year * 12 + issue_date.month)
lgd_input_df = helper_functions.customize_data_lgd(lgd_input_var_dict)

In [15]:
# make predictions
loan_data_preprocessed_lgd_ead = lgd_input_df.copy()
loan_data_preprocessed = loan_data_preprocessed_lgd_ead.copy()

loan_data_preprocessed = helper_functions.expected_loss_estimation(lgd_model_stage_1,lgd_model_stage_2,reg_ead,loan_data_preprocessed,loan_data_preprocessed_lgd_ead)

pred_dict = default_prediction(pd_input_df,reg_pd)
loan_data_preprocessed['PD'] = pred_dict['default_prob'][0][1]
# loan_data_preprocessed['Credit_Score'] = 
loan_data_preprocessed['EL'] = loan_data_preprocessed['PD'] * loan_data_preprocessed['LGD'] * loan_data_preprocessed['EAD']
loan_data_preprocessed['EL/funded_amnt'] = loan_data_preprocessed['EL']/loan_data_preprocessed['funded_amnt']
loan_data_preprocessed

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:MORTGAGE,home_ownership:NONE,home_ownership:OTHER,home_ownership:OWN,verification_status:Not Verified,verification_status:Source Verified,purpose:car,purpose:debt_consolidation,purpose:educational,purpose:home_improvement,purpose:house,purpose:major_purchase,purpose:medical,purpose:moving,purpose:other,purpose:renewable_energy,purpose:small_business,purpose:vacation,purpose:wedding,initial_list_status:w,term_int,emp_length_int,mths_since_issue_d,mths_since_earliest_cr_line,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,acc_now_delinq,total_rev_hi_lim,recovery_rate_st_1,recovery_rate_st_2,recovery_rate,LGD,CCF,EAD,PD,EL,EL/funded_amnt
0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36,2,51,147,100000,8,6,1000000,4,2,3,20,10,18,4,27,0,30000,1,0.157943,0.157943,0.842057,0.911785,91178.543658,0.966312,74191.004962,0.74191


In [23]:
pd_input_df

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,addr_state:NY,addr_state:OK_TN_MO_LA_MD_NC,addr_state:CA,addr_state:UT_KY_AZ_NJ,addr_state:AR_MI_PA_OH_MN,addr_state:RI_MA_DE_SD_IN,addr_state:GA_WA_OR,addr_state:WI_MT,addr_state:TX,addr_state:IL_CT,addr_state:KS_SC_CO_VT_AK_MS,addr_state:WV_NH_WY_DC_ME_ID,verification_status:Not Verified,verification_status:Source Verified,purpose:credit_card,purpose:debt_consolidation,purpose:oth__med__vacation,purpose:major_purch__car__home_impr,initial_list_status:w,term:36,emp_length:1,emp_length:2-4,emp_length:5-6,emp_length:7-9,emp_length:10,mths_since_issue_d:<38,mths_since_issue_d:38-39,mths_since_issue_d:40-41,mths_since_issue_d:42-48,mths_since_issue_d:49-52,mths_since_issue_d:53-64,mths_since_issue_d:65-84,int_rate:<9.548,int_rate:9.548-12.025,int_rate:12.025-15.74,int_rate:15.74-20.281,mths_since_earliest_cr_line:141-164,mths_since_earliest_cr_line:165-247,mths_since_earliest_cr_line:248-270,mths_since_earliest_cr_line:271-352,mths_since_earliest_cr_line:>352,inq_last_6mths:0,inq_last_6mths:1-2,inq_last_6mths:3-6,acc_now_delinq:>=1,annual_inc:20K-30K,annual_inc:30K-40K,annual_inc:40K-50K,annual_inc:50K-60K,annual_inc:60K-70K,annual_inc:70K-80K,annual_inc:80K-90K,annual_inc:90K-100K,annual_inc:100K-120K,annual_inc:120K-140K,annual_inc:>140K,dti:<=1.4,dti:1.4-3.5,dti:3.5-7.7,dti:7.7-10.5,dti:10.5-16.1,dti:16.1-20.3,dti:20.3-21.7,dti:21.7-22.4,dti:22.4-35,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [21]:
pred_dict = default_prediction(pd_input_df,reg_pd)

In [22]:
pred_dict

{'default_prediction': array([1]),
 'default_prob': array([[0.03368801, 0.96631199]])}

In [13]:
# predicting default for a customer
def default_prediction(input_df,model):

    pred_dict = {
        "default_prediction" : model.predict(input_df),
        "default_prob" : model.predict_proba(input_df)
    }

    return pred_dict

In [6]:
# input_dict = {key:0 for key in config.features_all}
input_df = pd.DataFrame({   'grade:A': 1,
                            'grade:B': 0,
                            'grade:C': 0,
                            'grade:D': 0,
                            'grade:E': 0,
                            'grade:F': 0,
                            'home_ownership:MORTGAGE': 0,
                            'home_ownership:NONE': 1,
                            'home_ownership:OTHER': 0,
                            'home_ownership:OWN': 0,
                            'verification_status:Not Verified': 0,
                            'verification_status:Source Verified': 1,
                            'purpose:car': 0,
                            'purpose:debt_consolidation': 0,
                            'purpose:educational': 0,
                            'purpose:home_improvement': 0,
                            'purpose:house': 1,
                            'purpose:major_purchase': 0,
                            'purpose:medical': 0,
                            'purpose:moving': 0,
                            'purpose:other': 0,
                            'purpose:renewable_energy': 0,
                            'purpose:small_business': 0,
                            'purpose:vacation': 0,
                            'purpose:wedding': 0,
                            'initial_list_status:w': 0,
                            'term_int': 3,
                            'emp_length_int': 22,
                            'mths_since_issue_d': 40,
                            'mths_since_earliest_cr_line': 50,
                            'funded_amnt': 100000,
                            'int_rate': 8,
                            'installment': 6,
                            'annual_inc': 10000000,
                            'dti': 20,
                            'delinq_2yrs': 1,
                            'inq_last_6mths': 3,
                            'mths_since_last_delinq': 12,
                            'mths_since_last_record': 23,
                            'open_acc': 7,
                            'pub_rec': 10,
                            'total_acc': 16,
                            'acc_now_delinq': 2,
                            'total_rev_hi_lim': 10000},index=[0])

loan_data_preprocessed_lgd_ead = input_df.copy()
loan_data_preprocessed = loan_data_preprocessed_lgd_ead.copy()

loan_data_preprocessed = helper_functions.expected_loss_estimation(lgd_model_stage_1,lgd_model_stage_2,reg_ead,loan_data_preprocessed,loan_data_preprocessed_lgd_ead)
loan_data_preprocessed

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:MORTGAGE,home_ownership:NONE,home_ownership:OTHER,home_ownership:OWN,verification_status:Not Verified,verification_status:Source Verified,purpose:car,purpose:debt_consolidation,purpose:educational,purpose:home_improvement,purpose:house,purpose:major_purchase,purpose:medical,purpose:moving,purpose:other,purpose:renewable_energy,purpose:small_business,purpose:vacation,purpose:wedding,initial_list_status:w,term_int,emp_length_int,mths_since_issue_d,mths_since_earliest_cr_line,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,acc_now_delinq,total_rev_hi_lim,recovery_rate_st_1,recovery_rate_st_2,recovery_rate,LGD,CCF,EAD
0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,22,40,50,100000,8,6,10000000,20,1,3,12,23,7,10,16,2,10000,1,0.906361,0.906361,0.093639,0.926806,92680.587195


#### Steps:
1. take input data for variables.
2. split data wrt models.
3. customize these var values.
4. run model to get predictions.

In [29]:
# storing input values of variables in a dictionary
var_data_dict = {
    'grade' : 'A',
    'home_ownership' : 'OWN',
    'addr_state' : 'NY',
    'verification_status' : 'Verifeid',
    'purpose' : 'education',
    'initial_list_status' : 'f',
    'term' : 36,
    'emp_length' : 2,
    'issue_d' : '2013-09-01',
    'int_rate' : 8,
    'earliest_cr_line' : '2005-09-01',
    'delinq_2yrs' : 2,
    'inq_last_6mths' : 3,
    'open_acc' : 18,
    'pub_rec' : 4,
    'total_acc' : 27,
    'acc_now_delinq' : 0,
    'total_rev_hi_lim' : 30000,
    'annual_inc' : 1000000,
    'dti' : 4,
    'mths_since_last_delinq' : 20,
    'mths_since_last_record' : 10,
    'funded_amnt' : 100000,
    'installment' : round(100000/36,2)
}

In [40]:
# splitting data wrt models
## for PD Model:
pd_input_var_dict = {key: var_data_dict[key] for key in pd_features if key in var_data_dict}
print(len(pd_features))
print(len(pd_input_var_dict.keys()))
## for LGD, EAD and EL
lgd_input_var_dict = {key: var_data_dict[key] for key in lgd_features if key in var_data_dict}
print(len(lgd_features))
print(len(lgd_input_var_dict.keys()))

17
17
23
23


In [41]:
from datetime import datetime
# customizing the input var values
## fro PD
pd_input_df = helper_functions.customize_data_pd(pd_input_var_dict)
## for LGD, EAD and EL
lgd_input_var_dict['term_int'] = lgd_input_var_dict.pop('term')
lgd_input_var_dict['emp_length_int'] = lgd_input_var_dict.pop('emp_length')
lgd_input_var_dict['mths_since_issue_d'] = lgd_input_var_dict.pop('issue_d')
issue_date = datetime.strptime(lgd_input_var_dict['mths_since_issue_d'], "%Y-%m-%d")
end_date = datetime.strptime('2017-12-01', "%Y-%m-%d")
lgd_input_var_dict['mths_since_issue_d'] = end_date.year * 12 + end_date.month - (issue_date.year * 12 + issue_date.month)
lgd_input_var_dict['mths_since_earliest_cr_line'] = lgd_input_var_dict.pop('earliest_cr_line')
issue_date = datetime.strptime(lgd_input_var_dict['mths_since_earliest_cr_line'], "%Y-%m-%d")
end_date = datetime.strptime('2017-12-01', "%Y-%m-%d")
lgd_input_var_dict['mths_since_earliest_cr_line'] = end_date.year * 12 + end_date.month - (issue_date.year * 12 + issue_date.month)
lgd_input_df = helper_functions.customize_data_lgd(lgd_input_var_dict)

AttributeError: 'int' object has no attribute 'isin'

In [39]:
input_df =  helper_functions.customdata(
    grade = "A",
    issue_d = '2013-09-01',
    int_rate = 8,
    annual_inc = 10000000
)

pred_dict = helper_functions.default_prediction(input_df)
print(pred_dict)

loan_data_preprocessed['PD'] = pred_dict['default_prob'][0][1]
loan_data_preprocessed['EL'] = loan_data_preprocessed['PD'] * loan_data_preprocessed['LGD'] * loan_data_preprocessed['EAD']
loan_data_preprocessed['EL/funded_amnt'] = loan_data_preprocessed['EL']/loan_data_preprocessed['funded_amnt']
loan_data_preprocessed

{'default_prediction': array([1]), 'default_prob': array([[0.02726491, 0.97273509]])}


,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:MORTGAGE,home_ownership:NONE,home_ownership:OTHER,home_ownership:OWN,verification_status:Not Verified,verification_status:Source Verified,purpose:car,purpose:debt_consolidation,purpose:educational,purpose:home_improvement,purpose:house,purpose:major_purchase,purpose:medical,purpose:moving,purpose:other,purpose:renewable_energy,purpose:small_business,purpose:vacation,purpose:wedding,initial_list_status:w,term_int,emp_length_int,mths_since_issue_d,mths_since_earliest_cr_line,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,acc_now_delinq,total_rev_hi_lim,recovery_rate_st_1,recovery_rate_st_2,recovery_rate,LGD,CCF,EAD,PD,EL,EL/funded_amnt
0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,22,40,50,100000,8,6,10000000,20,1,3,12,23,7,10,16,2,10000,1,0.906361,0.906361,0.093639,0.926806,92680.587195,0.972735,8441.929313,0.084419


In [19]:
loan_data_preprocessed['PD'][0]

0.9663119917365041